# Semantic Code Search using ALBERT and TRANSFORMERS - Part II.3
by:
### Maj Ashish Ahluwalia (21111073) ashisha21@iitk.ac.in
### Binay Kumar Suna (21111021) binayas21@iitk.ac.in
### Chabil Kansal (21111022) chabilk21@iitk.ac.in
### Shubham Sinha (21111409) shubhams21@iitk.ac.in


In [ ]:
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 4.0 MB 5.6 MB/s 
     |████████████████████████████████| 77 kB 4.8 MB/s 
     |████████████████████████████████| 895 kB 49.0 MB/s 
     |████████████████████████████████| 596 kB 45.6 MB/s 
     |████████████████████████████████| 6.6 MB 36.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 5.3 MB/s 


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.8.0


In [ ]:
import numpy as np
import math
import random
import torch
import csv
import pandas as pd
import transformers
from transformers import  AlbertConfig, TFAlbertModel
from transformers import AlbertTokenizer

In [ ]:
from pathlib import Path
import shutil
 
shutil.copy2('/content/drive/MyDrive/CS657 IR PROJECT/create_pretraining_data.py', './')
shutil.copy2('/content/drive/MyDrive/CS657 IR PROJECT/run_pretraining.py', './')
shutil.copy2('/content/drive/MyDrive/CS657 IR PROJECT/albert_base_3/assets/30k-clean.model', './')
shutil.copy2('/content/drive/MyDrive/CS657 IR PROJECT/albert_base_3/assets/30k-clean.vocab', './')
shutil.copy2('/content/drive/MyDrive/CS657 IR PROJECT/albert_base_3/assets/albert_config.json', './')
# shutil.copy2('/content/drive/MyDrive/CS657 IR PROJECT/tf_examples', './')
shutil.copy2('/content/drive/MyDrive/CS657 IR PROJECT/tf_examples2', './')
shutil.copy2('/content/drive/MyDrive/CS657 IR PROJECT/docstrings.txt', './')
shutil.copy2('/content/drive/MyDrive/CS657 IR PROJECT/albert_base_3/model/pytorch_model.bin', './')

'./pytorch_model.bin'

In [ ]:
config = AlbertConfig.from_pretrained('./', output_hidden_states=True)

model = TFAlbertModel.from_pretrained('./', config=config,  from_pt=True)
print(model.config)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFAlbertModel: ['predictions.decoder.bias', 'albert.embeddings.position_ids', 'predictions.LayerNorm.weight', 'predictions.bias', 'sop_classifier.classifier.bias', 'predictions.dense.weight', 'predictions.decoder.weight', 'sop_classifier.classifier.weight', 'predictions.LayerNorm.bias', 'predictions.dense.bias']
- This IS expected if you are initializing TFAlbertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFAlbertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on,

AlbertConfig {
  "_name_or_path": "./",
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "vocab_size": 30000
}



In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/CS657 IR PROJECT/train_sorted.csv')

In [ ]:
train_df.head()

,Unnamed: 0,index,nwo,path,function_name,lineno,original_function,function_tokens,docstring_tokens,url,function_tokens_count
0,137791,145692,geolib.py,pygeotools/lib/geolib.py,dd2dm,1,"def dd2dm(dd):\n """"""Convert decimal to degr...",dd dd float 3600 return,Convert decimal to degrees decimal minutes,https://github.com/dshean/pygeotools/blob/5ac7...,5
1,139491,159222,covar_plotter3.py,megaman/plotter/covar_plotter3.py,merge_keywords,1,"def merge_keywords(x,y):\n """"""Given two dic...",merge keywords copy update return,Given two dicts merge them into a new dict as ...,https://github.com/mmp2/megaman/blob/faccaf267...,5
2,60562,74099,fitz.py,fitz/fitz.py,Annot.setLineEnds,1,"def setLineEnds(self, start, end):\n """"...",checkparent self return fitz annot,setLineEnds ( self start end ),https://github.com/pymupdf/PyMuPDF/blob/917f2d...,5
3,62809,124310,aero.py,traffic/core/aero.py,vcas2mach,1,"def vcas2mach(cas, h):\n """""" CAS to Mach co...",cas tas cas tas return,CAS to Mach conversion,https://github.com/xoolive/traffic/blob/d1a887...,5
4,86012,189628,wcs.py,ginga/util/wcs.py,hmsToDeg,1,"def hmsToDeg(h, m, s):\n """"""Convert RA hour...",hmstodeg return degperhmshour degperhmsmin deg...,Convert RA hours minutes seconds into an angle...,https://github.com/ejeschke/ginga/blob/a78c893...,5


In [ ]:
print(len(train_df['docstring_tokens'].values))

161098


In [ ]:
print(len(train_df['index'].values))

161098


In [ ]:
albert_tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2') 

Downloading:   0%|          | 0.00/742k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/684 [00:00<?, ?B/s]

In [ ]:
avg_embeddings = []
temp = 0
for count,item in enumerate(train_df['docstring_tokens'].values): #traverse thorugh all train data set docstrings
    e = albert_tokenizer.encode(item, max_length=512)
    input = tf.constant(e)[None, :]  # Batch size 1 
    output = model(input)
    v = [0]*768
    temp += 1
    for i in range(1, len(input[0])-1):
        v = v + output[0][0][i].numpy()  # generate sentence vectors by averaging the word vectors
    avg_embeddings.append(v/len(input[0])) #append all sentence vectors into a list
    if(temp == 1000):
        print(temp)
        temp = 0


1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000


In [ ]:
# Save the sentence embeddings in a .tsv file
with open("/content/drive/MyDrive/CS657 IR PROJECT/avg_embeddings.tsv", "w+", newline='') as my_csv:
    csvWriter = csv.writer(my_csv, delimiter='\t')
    csvWriter.writerows(avg_embeddings)